In [1]:
import pandas as pd
import sqlite3

# Connect to SQLite database (creates olist.db if it doesn’t exist)
conn = sqlite3.connect('olist.db')

# Dictionary of CSV files and their corresponding table names
datasets = {
    'olist_orders_dataset.csv': 'orders',
    'olist_products_dataset.csv': 'products',
    'olist_sellers_dataset.csv': 'sellers',
    'product_category_name_translation.csv': 'product_category_translation',
    'olist_geolocation_dataset.csv': 'geolocation',
    'olist_order_items_dataset.csv': 'order_items',
    'olist_order_payments_dataset.csv': 'order_payments',
    'olist_order_reviews_dataset.csv': 'order_reviews',
    'olist_customers_dataset.csv': 'customers'
}

# Load each CSV into SQLite
for csv_file, table_name in datasets.items():
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Loaded {csv_file} into {table_name} table.")

# Close the connection
conn.close()

Loaded olist_orders_dataset.csv into orders table.
Loaded olist_products_dataset.csv into products table.
Loaded olist_sellers_dataset.csv into sellers table.
Loaded product_category_name_translation.csv into product_category_translation table.
Loaded olist_geolocation_dataset.csv into geolocation table.
Loaded olist_order_items_dataset.csv into order_items table.
Loaded olist_order_payments_dataset.csv into order_payments table.
Loaded olist_order_reviews_dataset.csv into order_reviews table.
Loaded olist_customers_dataset.csv into customers table.


In [2]:
conn = sqlite3.connect('olist.db')

In [3]:
query1 = """
WITH customer_totals AS (
    SELECT c.customer_unique_id, SUM(oi.price) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    GROUP BY c.customer_unique_id
)
SELECT customer_unique_id, total_spent
FROM customer_totals
ORDER BY total_spent DESC
LIMIT 5;
"""
print("Top 5 customers by total order value:")
print(pd.read_sql_query(query1, conn))

Top 5 customers by total order value:
                 customer_unique_id  total_spent
0  0a0a92112bd4c708ca5fde585afaa872      13440.0
1  da122df9eeddfedc1dc1f5349a1a690c       7388.0
2  763c8b1c9c68a0229c42c9fc6f662b93       7160.0
3  dc4802a71eae9be1dd28f5d788ceb526       6735.0
4  459bef486812aa25204be022145caa62       6729.0


In [4]:
query2 = """
SELECT strftime('%Y-%m', order_purchase_timestamp) AS month,
       COUNT(order_id) AS order_count,
       LAG(COUNT(order_id), 1) OVER (ORDER BY strftime('%Y-%m', order_purchase_timestamp)) AS prev_month_orders,
       (COUNT(order_id) - LAG(COUNT(order_id), 1) OVER (ORDER BY strftime('%Y-%m', order_purchase_timestamp))) AS order_growth
FROM orders
GROUP BY month
ORDER BY month;
"""
print("\nMonthly order growth:")
print(pd.read_sql_query(query2, conn))


Monthly order growth:
      month  order_count  prev_month_orders  order_growth
0   2016-09            4                NaN           NaN
1   2016-10          324                4.0         320.0
2   2016-12            1              324.0        -323.0
3   2017-01          800                1.0         799.0
4   2017-02         1780              800.0         980.0
5   2017-03         2682             1780.0         902.0
6   2017-04         2404             2682.0        -278.0
7   2017-05         3700             2404.0        1296.0
8   2017-06         3245             3700.0        -455.0
9   2017-07         4026             3245.0         781.0
10  2017-08         4331             4026.0         305.0
11  2017-09         4285             4331.0         -46.0
12  2017-10         4631             4285.0         346.0
13  2017-11         7544             4631.0        2913.0
14  2017-12         5673             7544.0       -1871.0
15  2018-01         7269             5673.0      

In [5]:
query3 = """
SELECT s.seller_state,
       AVG(julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp)) AS avg_delivery_days
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN sellers s ON oi.seller_id = s.seller_id
WHERE o.order_delivered_customer_date IS NOT NULL
GROUP BY s.seller_state
ORDER BY avg_delivery_days DESC;
"""
print("\nAverage delivery time by seller state:")
print(pd.read_sql_query(query3, conn))


Average delivery time by seller state:
   seller_state  avg_delivery_days
0            AM          47.840802
1            CE          17.885086
2            MA          17.737843
3            RO          17.411102
4            MT          14.745550
5            BA          13.845866
6            PI          13.789200
7            SC          13.569644
8            PA          13.427086
9            PR          13.383955
10           RN          13.018111
11           PE          12.912757
12           ES          12.876498
13           GO          12.827967
14           MG          12.796998
15           SE          12.702991
16           DF          12.527214
17           PB          12.522304
18           MS          12.327364
19           SP          12.277126
20           RJ          12.014197
21           RS          11.559426


In [6]:
conn.execute("UPDATE products SET product_category_name = 'Unknown' WHERE product_category_name IS NULL;")

In [7]:
conn.execute("ALTER TABLE orders ADD COLUMN order_year TEXT;")
conn.execute("ALTER TABLE orders ADD COLUMN order_month TEXT;")
conn.execute("UPDATE orders SET order_year = strftime('%Y', order_purchase_timestamp), order_month = strftime('%m', order_purchase_timestamp);")

In [8]:
query4 = "SELECT customer_unique_id, COUNT(*) FROM customers GROUP BY customer_unique_id HAVING COUNT(*) > 1;"
print(pd.read_sql_query(query4, conn))

                    customer_unique_id  COUNT(*)
0     00172711b30d52eea8b313a7f2cced02         2
1     004288347e5e88a27ded2bb23747066c         2
2     004b45ec5c64187465168251cd1c9c2f         2
3     0058f300f57d7b93c477a131a59b36c3         2
4     00a39521eb40f7012db50455bf083460         2
...                                ...       ...
2992  ff36be26206fffe1eb37afd54c70e18b         3
2993  ff44401d0d8f5b9c54a47374eb48c1b8         2
2994  ff8892f7c26aa0446da53d01b18df463         2
2995  ff922bdd6bafcdf99cb90d7f39cea5b3         3
2996  ffe254cc039740e17dd15a5305035928         2

[2997 rows x 2 columns]


In [9]:
conn.execute("""
DELETE FROM customers
WHERE rowid NOT IN (
    SELECT MIN(rowid) FROM customers GROUP BY customer_unique_id
);
""")